# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [11]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import joblib, pickle

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
subscription_id = '8a56ebc5-caa7-4a43-b8ed-1c1895e46051'
resource_group = 'aml-quickstarts-126131'
workspace_name = 'quick-starts-ws-126131'

ws = Workspace.get(name="quick-starts-ws-126131")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='mpg')
ds = dataset.to_pandas_dataframe()

ds = ds.replace('?',0)
ds.dropna(inplace=True)

ds.drop(columns=['car name'],inplace=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='regression',
    primary_metric='r2_score',
    training_data=ds,
    label_column_name='mpg',
    n_cross_validations=3)

In [6]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=============================

{'runId': 'AutoML_1fec89cb-437e-4064-96dc-ec1d7dd63aff',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-14T15:00:30.625988Z',
 'endTimeUtc': '2020-11-14T15:22:05.579211Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"8a56ebc5-caa7-4a43-b8ed-1c1895e46051","resource_group":"aml-quickstarts-126131","workspace_name":"quick-starts-ws-126131","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best,fitted = remote_run.get_output()
print(best)

Run(Experiment: AutoRanga,
Id: AutoML_1fec89cb-437e-4064-96dc-ec1d7dd63aff_44,
Type: None,
Status: Completed)


In [9]:
print(fitted)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [13]:
#TODO: Save the best model
filename = 'finalized_model.sav'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['finalized_model.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [62]:
# Register the model
# registered_model = remote_run.register_model(model_name='finalized_model.sav',
#                            tags={'area': 'mileage'},
#                            model_path='UPDATE WITH ACTUAL FOLDER e.g. ./training')

# OR

# loaded_model = pickle.load(open(filename,'rb'))
# loaded_model = joblib.load(filename)

registeredfinal = Model.register(workspace=ws,model_path='./training',model_name='registered')
# Model.register(workspace = ws, model_name = filename,)


print(registeredfinal.name, registeredfinal.id, registeredfinal.version, sep='\t')

#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-Tutorial')

env.python.conda_dependencies.add_pip_package('scikit-learn')

inferencefinal = InferenceConfig(entry_script='train_FINAL_11_12.py',environment=env)

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice

# deployfinal = AciWebservice.deploy_configuration(cpu_cores=3,memory_gb=10,enable_app_insights=True)
deployfinal = LocalWebservice.deploy_configuration(port=9050)
svcs = Model.deploy(workspace=ws,name='deployatlast',models=[registeredfinal],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Registering model registered
registered	registered:39	39
Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_082d46125b7151218bab353b1942f022
 ---> a618c566ff89
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 4921f3bf1614
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjhhNTZlYmM1LWNhYTctNGE0My1iOGVkLTFjMTg5NWU0NjA1MSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEyNjEzMSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEyNjEzMSIsIndvcmtzcGFjZUlkIjoiODRmYWZlMzctZWJkYS00MTcwLTk5M2QtZWM1NzRlMmVjYTI5In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 779c55135d53
 ---> 2b4887a85df1
Step 4/5 : RUN mv '/var/azureml-app/tmptlg8c1qp.py' /var/azureml-app/main.py
 --

ERROR - Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [63]:
print(svcs.get_logs)
# print(service.scoring_uri)
# print(service.swagger_uri)

TypeError: 'NoneType' object is not subscriptable

TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import requests
import json

# URL for the web service
scoring_uri = '<your web service URI>'

data = {"data":
            [
                18,
                4,
                122,
                87,
                2332,
                12,
                70,
                2
            ]
        
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

NameError: name 'key' is not defined

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()


### Convert model to ONNX format

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(fitted, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())